# Extract Sentences from Wikipedia
+ This notebook is used for collecting sentences that tell relationship between two entities from wikipedia using some dependency path pattern
+ **This notebook is fully valid under Owl3 machine (using the /scratch/data/wikipedia/full_text-2021-03-20 data)**

In [1]:
import re
from bs4 import BeautifulSoup
import pandas as pd
import sys
import wikipedia
import os
import random
import pickle
from wikipedia2vec import Wikipedia2Vec
import bz2
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import tqdm

sys.path.append('..')

from tools.BasicUtils import my_read, my_write, MyMultiProcessing, count_line
from tools.TextProcessing import (
                normalize_text, remove_brackets, my_sentence_tokenize, build_word_tree_v2, 
                my_sentence_tokenize, filter_specific_keywords, find_dependency_path_from_tree, find_span, nlp, 
                sent_lemmatize, exact_match
                )
from tools.DocProcessing import CoOccurrence, graph_load

from extract_wiki import SentenceFilter

from extract_wiki import (
    wikipedia_dir, wikipedia_entity_file, wikipedia_entity_norm_file, 
    wikipedia_keyword_file, wikipedia_token_file, wikipedia_wordtree_file, wikipedia_keyword_filtered_file, keyword_npmi_graph_file_v2, 
    save_path, keyword_occur_file, keyword_connection_graph_file, w2vec_dump_file, w2vec_keyword_file, w2vec_wordtree_file, w2vec_token_file, 
    w2vec_keyword2idx_file, path_test_file, test_path, cs_keyword_file, cs_token_file, cs_wordtree_file, cs_path_test_file, cs_raw_keyword_file, 
    collect_wiki_entity, get_sentence, keyword_count_file, filter_keyword_by_freq, line2note, note2line
)


# Create the multiprocessing object for future use
p = MyMultiProcessing(10)

# Generate the save dir
if not os.path.exists(save_path):
    os.mkdir(save_path)

if not os.path.exists(test_path):
    os.mkdir(test_path)

sub_folders = [sub for sub in os.listdir(wikipedia_dir)]
save_sub_folders = [os.path.join(save_path, sub) for sub in sub_folders]
wiki_sub_folders = [os.path.join(wikipedia_dir, sub) for sub in sub_folders]

wiki_files = []
save_sent_files = []
save_cooccur_files = []
save_cs_sent_files = []
save_selected_files = []

for save_dir in save_sub_folders:
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

for i in range(len(wiki_sub_folders)):
    files = [f for f in os.listdir(wiki_sub_folders[i])]
    wiki_files += [os.path.join(wiki_sub_folders[i], f) for f in files]
    save_sent_files += [os.path.join(save_sub_folders[i], f+'.dat') for f in files]
    save_cooccur_files += [os.path.join(save_sub_folders[i], f+'_co.dat') for f in files]
    save_cs_sent_files += [os.path.join(save_sub_folders[i], f+'_cs.dat') for f in files]
    save_selected_files += [os.path.join(save_sub_folders[i], f+'.tsv') for f in files]

# Get all files under wikipedia/full_text-2021-03-20

print('wiki sub folder example:', wiki_sub_folders[0])
print('save sub folder example:', save_sub_folders[0])
print('wiki file example:', wiki_files[0])
print('save sentence file example:', save_sent_files[0])
print('save cooccur file example:', save_cooccur_files[0])
print('save cs sentence file example:', save_cs_sent_files[0])
print('save selected sentence file example:', save_selected_files[0])

wiki sub folder example: ../../data/wikipedia/full_text-2021-03-20/BE
save sub folder example: data/extract_wiki/wiki_sent_collect/BE
wiki file example: ../../data/wikipedia/full_text-2021-03-20/BE/wiki_00
save sentence file example: data/extract_wiki/wiki_sent_collect/BE/wiki_00.dat
save cooccur file example: data/extract_wiki/wiki_sent_collect/BE/wiki_00_co.dat
save cs sentence file example: data/extract_wiki/wiki_sent_collect/BE/wiki_00_cs.dat
save selected sentence file example: data/extract_wiki/wiki_sent_collect/BE/wiki_00.tsv


## Collect wikipedia page titles as entities and generate keyword list

In [ ]:
# Collect wikipedia entities
output = p.run(collect_wiki_entity, wiki_files)
entity_list = []
for l in output:
    entity_list += l
my_write(wikipedia_entity_file, entity_list)

In [ ]:
# Get normalized wikipedia entities
normalized_entity = []
for kw in open(wikipedia_entity_file).readlines():
    eid, ent = kw.split('\t')
    normalized_entity.append('%s\t%s' % (eid, normalize_text(ent)))
my_write(wikipedia_entity_norm_file, normalized_entity)

In [ ]:
# Generate keyword list file
keywords = [remove_brackets(line.strip().split('\t')[1]) for line in open(wikipedia_entity_norm_file)]
keywords = [kw for kw in keywords if kw.split()]
keywords = filter_specific_keywords(keywords)
keywords = list(set(keywords))
my_write(wikipedia_keyword_file, keywords)

In [ ]:
# Build wordtree
build_word_tree_v2(wikipedia_keyword_file, wikipedia_wordtree_file, wikipedia_token_file)

## Collect sentences from wikipedia and select good sentences by path

In [ ]:
# Process the wikipedia page files to sentence only file (12 min)
wiki_sent_pair = [(wiki_files[i], save_sent_files[i]) for i in range(len(wiki_files))]
output = p.run(get_sentence, wiki_sent_pair)

In [ ]:
# Collect sentences from pages that have cs keywords as title ['collect_cs_pages']

In [ ]:
# Create the sentence filter
sf = SentenceFilter(wikipedia_wordtree_file, wikipedia_token_file)

In [ ]:
# Test of selecting sentences by path ['collect_sents']
def collect_sents(save_sent_file:str, save_selected_file:str):
    sents = my_read(save_sent_file)
    df = sf.list_operation(sents, use_id=True, keyword_only=True)
    df.to_csv(save_selected_file, sep='\t', index=False)

test_list = [(save_sent_files[i], '%d.tsv' % i) for i in range(5)]
test_output = p.run(collect_sents, test_list)

In [ ]:
# Get the keyword occurance (15 min) ['collect_kw_occur_from_selected']

In [ ]:
# Build connected graph from selected sentences(18 min) ['build_graph_from_selected']

In [ ]:
# Collect keyword cooccurance from sentence files (2 hours) ['collect_kw_occur_from_sents']

In [ ]:
# Filter the less frequent keywords using cooccurance files (8 min)
keyword_count = filter_keyword_by_freq(save_cooccur_file_list=save_cooccur_files)
with open(keyword_count_file, 'wb') as f_out:
    pickle.dump(keyword_count, f_out)

In [ ]:
f = {k : v for k, v in keyword_count.items() if v >= 300}
my_write(wikipedia_keyword_filtered_file, list(f.keys()))
print(len(f))

In [ ]:
'python' in f

In [ ]:
# Build connected graph from cooccurance files () ['build_graph_from_cooccur]

## Wikipedia2vec implementation

In [ ]:
# Load wikipedia2vec
with bz2.open(w2vec_dump_file) as f_in:
    w2vec = Wikipedia2Vec.load(f_in)

In [ ]:
# Create my_mention_dict, mapping keyword mention to wikipedia2vec entities
my_mention_dict = {}
for ent in w2vec.dictionary.entities():
    if ent.count < 50:
        continue
    kw = remove_brackets(normalize_text(ent.title))
    if kw not in my_mention_dict:
        my_mention_dict[kw] = [ent.index]
    else:
        my_mention_dict[kw].append(ent.index)
w2vec_kws = filter_specific_keywords(list(my_mention_dict.keys()))
my_write(w2vec_keyword_file, w2vec_kws)
build_word_tree_v2(w2vec_keyword_file, w2vec_wordtree_file, w2vec_token_file)
filter_keyword_from_w2vec = set(w2vec_kws)
my_mention_dict = {k:v for k, v in my_mention_dict.items() if k in filter_keyword_from_w2vec}
with open(w2vec_keyword2idx_file, 'wb') as f_out:
    pickle.dump(my_mention_dict, f_out)
len(my_mention_dict)

In [ ]:
# Create cs keyword that can be mapped to keyword mention based on cs_raw_keyword.txt
cs_raw_keyword = my_read(cs_raw_keyword_file)
cs_keyword = [kw for kw in cs_raw_keyword if kw in my_mention_dict]
my_write(cs_keyword_file, cs_keyword)
build_word_tree_v2(cs_keyword_file, cs_wordtree_file, cs_token_file)

In [ ]:
'smalltalk' in cs_keyword

In [ ]:
# Load CoOccurrence model
co = CoOccurrence(w2vec_wordtree_file, w2vec_token_file)
cs_co = CoOccurrence(cs_wordtree_file, cs_token_file)

In [ ]:
def collect_test_paths_all(test_file:str, co:CoOccurrence, posfix='.dat', disable_pbar:bool=False):
    # Build test data
    with open(test_file) as f_in:
        data = []
        for line_idx, line in enumerate(tqdm.tqdm(f_in.readlines(), disable=disable_pbar)):
            sent_note = line2note(test_file, line_idx, posfix=posfix)
            line = line.strip()
            co_kws = list(co.line_operation(sent_lemmatize(line)))
            if len(co_kws) < 2:
                continue
            certain_ent_list = []
            certain_ent_kw_list = []
            uncertain_ent_list = []
            uncertain_ent_kw_list = []
            for kw in co_kws:
                idxs = my_mention_dict[kw]
                if len(idxs) == 1:
                    certain_ent_kw_list.append(kw)
                    certain_ent_list.append(w2vec.dictionary.get_entity_by_index(idxs[0]))
                else:
                    uncertain_ent_kw_list.append(kw)
                    uncertain_ent_list.append([w2vec.dictionary.get_entity_by_index(idx) for idx in idxs])
            
            certain_ent_matrix = np.array([w2vec.get_vector(ent) for ent in certain_ent_list])
            uncertain_ent_matrix_list = [np.array([w2vec.get_vector(ent) for ent in ent_list]) for ent_list in uncertain_ent_list]
            pairs = []
            certain_len = len(certain_ent_list)
            uncertain_len = len(uncertain_ent_list)
            if certain_len >= 1:
                # Collect pairs between certain entities
                result = cosine_similarity(certain_ent_matrix, certain_ent_matrix) - np.identity(certain_len)
                for i in range(certain_len):
                    for j in range(i+1, certain_len):
                        pairs.append({'kw1':certain_ent_kw_list[i], 'kw2':certain_ent_kw_list[j], 'sim':float(result[i, j]), 'sent':sent_note, 
                            'kw1_ent':certain_ent_list[i].title, 
                            'kw2_ent':certain_ent_list[j].title})
                # Collect pairs between certain and uncertain entities
                for i in range(uncertain_len):
                    result = cosine_similarity(certain_ent_matrix, uncertain_ent_matrix_list[i])
                    for j in range(certain_len):
                        idx = np.argmax(result[j])
                        pairs.append({'kw1':uncertain_ent_kw_list[i], 'kw2':certain_ent_kw_list[j], 'sim':float(result[j, idx]), 'sent':sent_note, 
                            'kw1_ent':uncertain_ent_list[i][idx].title, 
                            'kw2_ent':certain_ent_list[j].title})
            if uncertain_len >= 2:
                # Collect pairs between uncertain entities
                for i in range(uncertain_len):
                    for j in range(i+1, uncertain_len):
                        result = cosine_similarity(uncertain_ent_matrix_list[i], uncertain_ent_matrix_list[j])
                        idx = np.argmax(result)
                        row = int(idx / result.shape[1])
                        col = idx % result.shape[1]
                        pairs.append({'kw1':uncertain_ent_kw_list[i], 'kw2':uncertain_ent_kw_list[j], 'sim':float(result[row, col]), 'sent':sent_note, 
                            'kw1_ent':uncertain_ent_list[i][row].title, 
                            'kw2_ent':uncertain_ent_list[j][col].title})
            doc = nlp(line)
            for item in pairs:
                kw1_spans = find_span(doc, item['kw1'], True)
                kw2_spans = find_span(doc, item['kw2'], True)
                for kw1_span in kw1_spans:
                    for kw2_span in kw2_spans:
                        path = find_dependency_path_from_tree(doc, kw1_span, kw2_span)
                        if not path:
                            continue
                        item['kw1_span'] = (kw1_span[0].i, kw1_span[-1].i)
                        item['kw2_span'] = (kw2_span[0].i, kw2_span[-1].i)
                        item['path'] = path
                        data.append(item.copy())
        if not disable_pbar:
            print(len(data))
        return pd.DataFrame(data=data, columns=['sim', 'kw1', 'kw1_span', 'kw1_ent', 'kw2', 'kw2_span', 'kw2_ent', 'sent', 'path'])

In [ ]:
# Collect data using general wikipedia keywords
wiki_path_test_df = pd.concat([collect_test_paths_all(file, co) for file in save_sent_files[:3]], ignore_index=True)
wiki_path_test_df.to_csv(path_test_file, sep='\t', index=False)
print(len(wiki_path_test_df))

In [ ]:
# Collect data using general wikipedia keywords
# line_per_file = [count_line(f) for f in save_cs_sent_files[:1100]]
# print(sum(line_per_file))
cs_path_test_df = pd.concat([collect_test_paths_all(file, cs_co, posfix='_cs.dat', disable_pbar=True) for file in tqdm.tqdm(save_cs_sent_files[:1100])], ignore_index=True)
cs_path_test_df.to_csv(cs_path_test_file, sep='\t', index=False)
print(len(cs_path_test_df))

In [ ]:
'programming language' in my_mention_dict

In [ ]:
for idx in my_mention_dict['programming language']:
    print(w2vec.dictionary.get_item_by_index(idx))

In [ ]:
w2vec.most_similar_by_vector(w2vec.get_entity_vector('Python (programming language)'), 20)

In [ ]:
cosine_similarity(w2vec.get_entity_vector('Political movement').reshape(1, -1), w2vec.get_entity_vector('Hierarchy').reshape(1, -1))

In [ ]:
e = w2vec.get_entity('The World (radio program)')

In [ ]:
print(e.count)
print(e.doc_count)

In [ ]:
note2line('AA:00:0', posfix='_cs.dat')

In [2]:
def collect_path_test_data_sim_based(df:pd.DataFrame, sim_threshold:float=0.5):
    # Find all keyword cooccurrence and keep the ones that are similar in wikipedia2vec
    return df[df['sim'] >= sim_threshold]
    
# def collect_path_test_data_npmi_based(df:pd.DataFrame, npmi_threshold:float=0.1):
#     # Find all keyword cooccurrence and keep the ones that are similar in wikipedia2vec
#     return df[df['npmi'] >= npmi_threshold]
    
def map_path_pattern_to_test_data(df:pd.DataFrame, paths:list):
    def match_path_pattern(path:str):
        for pp in paths:
            if exact_match(pp, path):
                return pp
        return ''
    df['pattern'] = df.apply(lambda x: match_path_pattern(x['path']), axis=1)
    # For each remained pair, get the dep path

In [5]:
wiki_test_df = collect_path_test_data_sim_based(wiki_path_test_df, 0.0)
cs_test_df = collect_path_test_data_sim_based(cs_path_test_df, 0.0)

In [6]:
patterns = [
    r'i_nsubj attr( prep pobj)*( compound){0,1}', 
    # r'i_nsubj attr( prep pobj)*', 
    r'i_nsubj( conj)* dobj( acl prep pobj( conj)*){0,1}', 
    r'i_nsubj( prep pobj)+( conj)*', 
    r'i_nsubj advcl dobj( acl attr){0,1}', 
    r'appos( conj)*', 
    r'appos acl prep pobj( conj)*', 
    r'i_nsubjpass( conj)*( prep pobj)+( conj)*', 
    r'i_nsubjpass prep pobj acl dobj', 
    r'i_dobj prep pobj( conj)*'
    # 'acl prep pobj( conj)*'
]
map_path_pattern_to_test_data(wiki_test_df, patterns)
map_path_pattern_to_test_data(cs_test_df, patterns)

/tmp/ipykernel_3923/1819995245.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pattern'] = df.apply(lambda x: match_path_pattern(x['path']), axis=1)


In [7]:
def analysis_path_result_sim_based(df:pd.DataFrame, paths:list):
    summary_df = pd.DataFrame(columns=['path', 'cnt', 'ratio', 'avg_sim'])
    for pp in paths:
        sub_df = df[df['pattern'] == pp]
        summary_df = summary_df.append({
            'path' : pp,
            'cnt' : len(sub_df),
            'ratio' : len(sub_df) / len(df),
            'avg_sim' : sum(sub_df['sim']) / len(sub_df) if len(sub_df) else 0
        }, ignore_index=True)
    summary_df = summary_df.append({
        'path' : 'general',
        'cnt' : len(df),
        'ratio' : 1,
        'avg_sim' : sum(df['sim']) / len(df) if len(df) else 0
    }, ignore_index=True)
    return summary_df

# def analysis_path_result_npmi_based(df:pd.DataFrame, paths:list):
#     summary_df = pd.DataFrame(columns=['path', 'cnt', 'ratio', 'avg_npmi'])
#     for pp in paths:
#         sub_df = df[df['pattern'] == pp]
#         summary_df = summary_df.append({
#             'path' : pp,
#             'cnt' : len(sub_df),
#             'ratio' : len(sub_df) / len(df),
#             'avg_npmi' : sum(sub_df['npmi']) / len(sub_df) if len(sub_df) else 0
#         }, ignore_index=True)
#     summary_df = summary_df.append({
#         'path' : 'general',
#         'cnt' : len(df),
#         'ratio' : 1,
#         'avg_npmi' : sum(df['npmi']) / len(df) if len(df) else 0
#     }, ignore_index=True)
#     return summary_df

In [8]:
analysis_path_result_sim_based(wiki_test_df, patterns)

,path,cnt,ratio,avg_sim
0,"i_nsubj attr( prep pobj)*( compound){0,1}",912,0.002138,0.345963
1,"i_nsubj( conj)* dobj( acl prep pobj( conj)*){0,1}",1351,0.003167,0.341431
2,i_nsubj( prep pobj)+( conj)*,2485,0.005824,0.308623
3,"i_nsubj advcl dobj( acl attr){0,1}",191,0.000448,0.334822
4,appos( conj)*,1228,0.002878,0.369404
5,appos acl prep pobj( conj)*,30,0.000070,0.323274
6,i_nsubjpass( conj)*( prep pobj)+( conj)*,1686,0.003952,0.312614
7,i_nsubjpass prep pobj acl dobj,9,0.000021,0.329141
8,i_dobj prep pobj( conj)*,1769,0.004146,0.306733
9,general,426647,1.000000,0.319292


In [9]:
analysis_path_result_sim_based(cs_test_df, patterns)

,path,cnt,ratio,avg_sim
0,"i_nsubj attr( prep pobj)*( compound){0,1}",584,0.005262,0.484567
1,"i_nsubj( conj)* dobj( acl prep pobj( conj)*){0,1}",605,0.005451,0.453143
2,i_nsubj( prep pobj)+( conj)*,674,0.006073,0.472138
3,"i_nsubj advcl dobj( acl attr){0,1}",76,0.000685,0.442264
4,appos( conj)*,314,0.002829,0.465873
5,appos acl prep pobj( conj)*,6,0.000054,0.487080
6,i_nsubjpass( conj)*( prep pobj)+( conj)*,618,0.005568,0.477143
7,i_nsubjpass prep pobj acl dobj,6,0.000054,0.472865
8,i_dobj prep pobj( conj)*,355,0.003199,0.480274
9,general,110986,1.000000,0.453584


In [ ]:
def collect_example_sent_for_pattern(df:pd.DataFrame, path:str, num:int=30, posfix:str='.dat'):
    sub_df = df[df['pattern'] == path]
    num = min(len(sub_df), num)
    sub_df = sub_df[:num]
    sub_df['sent'] = sub_df.apply(lambda x: note2line(x['sent'], posfix=posfix).strip(), axis=1)
    return sub_df

for patt in patterns:
    temp_df = collect_example_sent_for_pattern(wiki_test_df, patt)
    temp_df.to_csv('%s.tsv' % (patt[:10] if len(patt) >= 10 else patt), index=False, sep='\t')

## Demo

In [ ]:
# Load keyword occur dict which has occurance record for all keywords in selected sentences
with open(keyword_occur_file, 'rb') as f_in:
    keyword_occur = pickle.load(f_in)

In [ ]:
# Load keyword connection graph in selected sentences
with open(keyword_connection_graph_file, 'rb') as f_in:
    keyword_connection_graph = pickle.load(f_in)

In [ ]:
# Load keyword count file
with open(keyword_count_file, 'rb') as f_in:
    keyword_count = pickle.load(f_in)

In [ ]:
# Load mention to index file
with open(w2vec_keyword2idx_file, 'rb') as f_in:
    my_mention_dict = pickle.load(f_in)

In [4]:
# Load path test data (pd.DataFrame)
wiki_path_test_df = pd.read_csv(open(path_test_file), sep='\t')
cs_path_test_df = pd.read_csv(open(cs_path_test_file), sep='\t')

In [ ]:
from collections import Counter
def reverse_path(path:str):
    path = path.split()
    r_path = ' '.join(['i_' + token if token[:2] != 'i_' else token[2:] for token in reversed(path)])
    return r_path

# sub_df = wiki_path_test_df[wiki_path_test_df['sim'] > 0.5]
sub_df = cs_path_test_df[cs_path_test_df['npmi'] > 0.15]
sub_df['pick'] = sub_df.apply(lambda x: 1 if 'nsubj' in x['path'] else 0, axis=1)
sub_df = sub_df[sub_df['pick'] > 0]
sub_df['path'] = sub_df.apply(lambda x: reverse_path(x['path']) if 'i_nsubj' not in x['path'] else x['path'], axis=1)
# sub_df['sent'] = sub_df.apply(lambda x: note2line(x['sent']).strip(), axis=1)
sub_df['sent'] = sub_df.apply(lambda x: note2line2(x['sent']).strip(), axis=1)
c = Counter(sub_df['path'].to_list())
# for k in c.keys():
#     if c[k] < 5:
#         c[k] = 0
max_cnt = c.most_common(1)[0][1]
log_max_cnt = np.log(max_cnt)
def cal_score(path:str):
    cnt = max(c.get(path), 1)
    return np.log(cnt) / log_max_cnt
sub_df['score'] = sub_df.apply(lambda x: cal_score(x['path']), axis=1)
# sub_df.to_csv('nsubj.tsv', index=False, sep='\t', columns=['sim', 'kw1', 'kw1_span', 'kw1_ent', 'kw2', 'kw2_span', 'kw2_ent', 'sent', 'path', 'score'])
sub_df.to_csv('nsubj_cs.tsv', index=False, sep='\t', columns=['npmi', 'kw1', 'kw1_span', 'kw2', 'kw2_span', 'sent', 'path', 'score'])

In [ ]:
c.most_common(10)

In [ ]:
# Load keyword connection graph in cooccurance files
with open(keyword_npmi_graph_file_v2, 'rb') as f_in:
    keyword_npmi_graph = pickle.load(f_in)
for k, v in keyword_npmi_graph[1].items():
    print(k)
    print(v)
    break

In [ ]:
# Demo function: find all the sentences that two keywords co-occur in selected sentences
def find_sentences(keyword_dict:dict, kw1:str, kw2:str):
    kw1_occur = keyword_dict.get(kw1)
    kw2_occur = keyword_dict.get(kw2)
    sents = pd.DataFrame(columns=['head', 'head_norm', 'head_span', 'tail', 'tail_norm', 'tail_span', 'sent', 'path'])
    if not kw1_occur or not kw2_occur:
        return sents
    co_occur = kw1_occur & kw2_occur
    file_dict = {}
    for occur in co_occur:
        sub_file, line_idx = occur.rsplit(':', 1)
        if sub_file not in file_dict:
            file_dict[sub_file] = []
        file_dict[sub_file].append(int(line_idx))
    for f, lines in file_dict.items():
        sentence_in_file = my_read(os.path.join(save_path, f.replace(':', '/wiki_')+'.dat'))
        records = my_read(os.path.join(save_path, f.replace(':', '/wiki_')+'.tsv'))
        for idx in lines:
            record = records[idx].split('\t')
            sent = sentence_in_file[int(record[6])]
            sents = sents.append({  'head':record[0],
                                    'head_norm':record[1],
                                    'head_span':record[2],
                                    'tail':record[3],
                                    'tail_norm':record[4],
                                    'tail_span':record[5],
                                    'sent':sent,
                                    'path':record[7]}, ignore_index=True)
    return sents

In [ ]:
df = find_sentences(keyword_occur, 'python', 'programming language')
df.to_csv('sents.tsv', sep='\t', index=False)

In [ ]:
'decision tree' in keyword_occur

In [ ]:
len(keyword_occur['machine learning'])

In [ ]:
kw1 = 'data mining'
kw2 = 'machine learning'
doc = nlp('Data mining is a process of extracting and discovering patterns in large data sets involving methods at the intersection of machine learning, statistics, and database systems.'.lower())
kw1_span = find_span(doc, kw1)
kw2_span = find_span(doc, kw2)
find_dependency_path_from_tree(doc, kw1_span[0], kw2_span[0])
# print(len(kw1_span))
# print(len(kw2_span))

In [ ]:
data = keyword_connection_graph.neighbors('decision tree')
my_write('neighbors.txt', list(data))

## Online operations

In [ ]:
def collect_sents_from_wiki_page(page:wikipedia.WikipediaPage):
    remove_list = ['See also', 'References', 'Further reading', 'Sources', 'External links']
    dic = {sec : page.section(sec) for sec in page.sections}
    dic['summary'] = page.summary
    sents = []
    section_list = list(dic.keys())
    while len(section_list) > 0:
        section = section_list.pop()
        if section in remove_list:
            continue
        section_text = dic[section]
        if not section_text:
            continue
        # processed_text = clean_text(section_text)
        processed_text = ' '.join(section_text.lower().split())
        temp_sents = my_sentence_tokenize(processed_text, True)
        sents += temp_sents
    return list(sents)

def collect_entity_from_wiki_page(page:wikipedia.WikipediaPage):
    return [text.lower() for text in page.links]

def collect_keyword_from_wiki_page(page:wikipedia.WikipediaPage):
    soup = BeautifulSoup(page.html(), 'html.parser')
    main_block = soup.find('div', class_='mw-parser-output')
    keywords = set([l.text.lower() for l in main_block.findAll('a') if re.match(r'^(<a href="/wiki/)', str(l))])
    return keywords



In [ ]:
keyword = 'python'

p = wikipedia.page(keyword)
if p is not None:
    sents = collect_sents_from_wiki_page(p)
    keywords = collect_keyword_from_wiki_page(p)
    print('sentences collected')
    my_write('%s.txt' % keyword, sents)
    my_write('%s_kw.txt' % keyword, keywords)
    df = filter_by_path(sents)
    df.to_csv('%s_out.tsv' % keyword, sep='\t', index=False)

    dff = df[df.apply(lambda x: str(x['head']) in keywords and str(x['tail']) in keywords, axis=1)]
    dff.to_csv('%s_out_f.tsv' % keyword, sep='\t', index=False)

In [ ]:
df['wanted'] = df.apply(lambda x: str(x['head']) in keywords, axis=1)

In [ ]:
df.head()

In [ ]:
dff.head()

In [ ]:
len(dff)